# Debate as seen through the prompts of LLMs

OpenAI has been leading the way in AI development, expanding the horizons of machine capabilities. Their work on Large Language Models (LLMs) has been groundbreaking, allowing machines to produce human-like text, answer queries, translate languages, and even write code. With the advent of the short revolution, new LLMs have emerged, such as Mistral AI, Gemini, and Claude-3, often surpassing their predecessors in various fields. Their performance is evaluated through benchmarks like the Massive Multitasks Language Understanding (MMLU) designed to measure the knowledge and problem-solving abilities of language models across a wide range of subjects, from STEM fields to the humanities and social sciences; TriviaQA dataset consisting of a comprehensive reading comprehension dataset that contains over 650,000 question-answer-evidence triples; and many others.  However, these tests may not fully capture an AI's capabilities, as real-world scenarios are often more complex and interactive than benchmark tests.

Unlike traditional metrics and tests used to evaluate LLMs, I was wondering whether we could assess AI by using AI: allowing AI models to interact directly with each other and evaluate their own responses and those of their peers. Inspired by debates used as a training framework to shape and challenge the minds of future leaders, lawyers, or judges (see e.g. [The importance of debate](https://oxfordsummercourses.com/articles/the-importance-of-debate/) or the [Debating society germany](https://www.schoolsdebate.de/index.php/about-us/our-competitions)), I designed a simple framework for AI to challenge each other. This framework consists of four parts: topic generation, debating, judging, and voting. In each part, roles will be assigned to LLMs using COSTAR prompts. The results format will be assessed, and if it doesn't fit the task, it will be rescheduled.

## Article goal:

In this article we will focus on the results of AI and aim to answer the questions

- Do LLMs perform better in debates when they choose their own topics?
- Are LLMs more effective in the proposing or opposing team?
- Do they judge a debate impartially; i.e., do they not favor themselves?
- Can their judgment persuade another AI?

## Short summary of LLM used
**OpenAI - ChatGPT-4:** GPT-4, the latest model from OpenAI, showcases several significant improvements over its predecessor, GPT-3.5. Its key strengths include enhanced language understanding and generation, enabling it to comprehend and produce various dialects and emotional nuances, as well as create more coherent and creative content. GPT-4 also demonstrates superior reasoning and problem-solving capabilities, tackling complex mathematical and scientific problems with ease. Its multimodal capabilities set it apart, as it can analyze and comment on images and graphics. With an increased scale and capacity, GPT-4 caters to long-form content creation, extended conversations, and document analysis. Moreover, its safety and alignment have been improved, making it more reliable in providing factual responses and refusing disallowed content. Lastly, GPT-4's advanced programming abilities make it a valuable resource for software developers.

Benchmarks: https://openai.com/research/gpt-4

**MistralAI:** This model showcases exceptional capabilities, such as efficient processing through a sparse mixture of experts, dynamic expert utilization for nuanced responses, and the ability to handle a context of 32k tokens. MistralAI's emphasis on open technology leadership, strong financial backing, and commitment to efficiency in AI solutions further solidify its position in the AI market. Additionally, MistralAI offers a range of products tailored to different needs, from cost-effective endpoints like Mistral-tiny to more robust offerings like Mistral-medium.

Benchmarks: https://docs.mistral.ai/platform/endpoints/

**Gemini-Pro:** Gemini-Pro showcases several notable improvements and features compared to its predecessors. Its key strengths include enhanced performance and efficiency, achieving comparable quality to larger models while using fewer computational resources. The model introduces a breakthrough in long-context understanding, processing up to 1 million tokens, which is the longest among any large-scale foundation models. Gemini-Pro also boasts multimodal capabilities, supporting both text and image inputs and comprehending 38 languages. With a focus on safety and alignment, Google has conducted extensive ethics and safety testing for responsible deployment. Additionally, Gemini-Pro offers faster inference speed, potentially leading to real-time latency gains, and demonstrates strong abilities in following simple instructions.

Benchmarks: https://blog.google/technology/ai/google-gemini-ai/#performance

**Claude-3:** Claude-3, the AI model from Anthropic, demonstrates more natural, human-like language abilities, engaging in coherent, creative, and nuanced conversations. The model outperforms competitors like GPT-4 in IQ tests and excels in mathematics, information retrieval, and other benchmarks. Claude-3 also features multimodal capabilities, enabling it to analyze and comment on images and graphics. nthropic has prioritized safety and alignment, making Claude-3 more reliable and less prone to harmful outputs. Lastly, the model's versatility allows it to tackle a wide range of tasks, from creative writing to academic-style analysis.

Benchmarks: https://www.anthropic.com/news/claude-3-family


# An AI Debate

## Short summary of the AI-debate flow:

* Each AI has generated 10 topics
* Pair of AI has been debated on their topics and the one of Gemini for 4 round in each team opposing or proposing.
    * MistralAI vs ChatGPT-4 
    * MistralAI vs Claude-3-Opus
* Each AI has judge the debate
* Each AI has voted for the best judgement

## Summary of the data generated

In [ ]:
from IPython.display import SVG
SVG(filename='AiDebate.drawio.svg')

## Score normalisation

Similarly to teachers or universities differing into their grading scales, I anticipated that LLMs differs in their grading scales as well. To compensate for this effect the scores as been normalised by using the minimum and maximum score for each judge and category.


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import HBox, VBox
from IPython.display import DisplayObject
from ai_debater.io_database import IODataBase
from plotly.figure_factory import create_scatterplotmatrix

#Some constant
neutral_judge = 'GeminiChatter|gemini-pro'
teams_of_interests = ['Claude3AiChatter|claude-3-opus-20240229','MistralAIChatter|mistral-large-latest','OpenAIChatter|gpt-4']


def get_judegement_groups(judgements, judge_of_interest, neutral_judge = neutral_judge, columngroup = 'is_team_topic_creator'):
    selected_judgements = judgements.loc[judgements.judge_entity == judge_of_interest].copy()
    selected_judgements['is_neutral_judge'] = False
    neutral_judgements =  judgements.loc[judgements.discourse_id.isin(selected_judgements.discourse_id)].copy()
    neutral_judgements = neutral_judgements.loc[neutral_judgements.judge_entity == neutral_judge]
    neutral_judgements['is_neutral_judge'] = True
    selected_judgements = selected_judgements.set_index(['discourse_id','Categories'])
    neutral_judgements = neutral_judgements.set_index(['discourse_id','Categories'])
    neutral_judgements[columngroup] = selected_judgements[columngroup]
    judgements2plot = pd.concat([selected_judgements, neutral_judgements])
    judgements2plot = judgements2plot.reset_index()
    def create_group(x,columngroup=columngroup):
        if columngroup == "is_team_topic_creator":
            if x.is_neutral_judge:
                y = '(1)' if x[columngroup] else '(2)'
                return f'Neutral judges group {y}'
            else:
                y = '(1): judging own topic' if x[columngroup] else '(2): judging topic of others'
                return f'Group {y}'
        elif columngroup == "is_team_self_judging":
            if x.is_neutral_judge:
                y = '(1)' if x[columngroup] else '(2)'
                return f'Neutral judges group {y}'
            else:
                y = '(1): self judgement' if x[columngroup] else '(2): judging others'
                return f'Group {y}'
        else:
            raise NameError("Not Implemented")
    judgements2plot['Judging groups'] = judgements2plot.apply(create_group, axis=1)
    return judgements2plot, selected_judgements, neutral_judgements

def plot_analyses(judgements, judge_of_interest = 'Claude3AiChatter|claude-3-opus-20240229', columngroup = 'is_team_topic_creator'):
    judgements2plot, selected_judgements, neutral_judgements = get_judegement_groups(judgements, judge_of_interest, columngroup=columngroup)
    fig1 = px.box(judgements2plot, x="Categories", y="Normalised_score", color="Judging groups")

    score_diff = selected_judgements.Normalised_score-neutral_judgements.Normalised_score
    score_diff = score_diff.to_frame()
    score_diff[columngroup] = selected_judgements[columngroup]
    score_diff = score_diff.reset_index()
    fig2 = px.box(score_diff, x="Categories", y="Normalised_score", color=columngroup)

    #fig = go.Figure(data = fig1.data + fig2.data)
    #fig.show()
    fw1 = go.FigureWidget(data=fig1.data)
    fw1.update_layout(
        title_text=f"<b>{judge_of_interest}</b><br>Score from participating judge and neutral judge",
        yaxis_title='Judge Score',
        boxmode='group' # group together boxes of the different traces for each value of x
    )
    fw2 = go.FigureWidget(data=fig2.data)
    fw2.update_layout(
        title_text="<br>Relative scoring",
        yaxis_title='Relative Score',
        boxmode='group' # group together boxes of the different traces for each value of x
    )
    fig_subplots=  HBox([fw1, fw2])
    return fig_subplots #this line displays the subplots in the Jupyter Notebook


In [ ]:
# Fetch the data
result_manager = IODataBase('results/dataset_v2.db')
judgements = result_manager.load_judgements_for_analyses()
judgements.Categories = judgements.Categories.str.replace('And','<br>')

# Score overview

We may first wonder whether we have a clear winning model that across categories and judges manage to consequently get a higher score in the debate. To answer this question we can look at the median score by categories and judges for the participating models. We can see in the polar plot below, that we don't have a clear winner, because the model getting the highest score seems to depend at least on the model judging the debate.

In [ ]:

results = judgements.copy()
results.judge_entity = results.judge_entity.str.replace('|','<br>')
judges = results.judge_entity.unique()
teams = results.team_entity.unique()
colors = {t:c for t,c in zip(teams, ["red","blue","green"])}
fig = make_subplots(rows=1, cols=len(judges), subplot_titles=judges, specs=[[{"type": "polar"}]*len(judges)],
horizontal_spacing=0.1)
for col_i, judge_name in enumerate(judges):
    df = results.loc[results.judge_entity==judge_name].copy()
    for team_entity, team_result in df.groupby('team_entity'):
        team_result = team_result.reset_index()
        team_result = team_result.groupby("Categories").Normalised_score.median()
        fig.add_trace(go.Scatterpolar(
            name=team_entity,
            legendgroup=team_entity,
            r=team_result.values,
            theta=team_result.index,
            marker=dict(color=colors[team_entity]),
            fill='toself'), row=1, col=col_i+1)
# Deduplicate legend
# see: https://stackoverflow.com/questions/26939121/how-to-avoid-duplicate-legend-labels-in-plotly-or-pass-custom-legend-labels
names = set()

fig.for_each_trace(
    lambda trace:
        trace.update(showlegend=False)
        if (trace.name in names) else names.add(trace.name))
#
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="top",
    y=0.0,
    xanchor="right",
    x=1
))
fig.show()

## Do LLM debate better on their own topics?
The debate were primed with information such as subject and rational which was created by the LLMs. This priming formulated by different AI may affect the ability of the AI to engage within the discourse. This may lead to a bias in the LLM ability to debate, or intuitively an AI debating on their own topic have a greater change to obtain a high score than debating on an opponent or neutral topic.

We can look at this effect by comparing the score of the AI participating in the debate and the score of a neutral AI (which is in this case Gemini). The neutral AI never participated in the debate, i.e. it never assumed the role of a debater, but did generate topics and debaters. For the comparison we consider two overarching groups: The neutral judge and the participating judge. The participating judge can be either judging its own generated topic (Group 1) or topic from another LLMs (Group 2). The scores are therefore grouped into: The participating judge judging Group 1 (self judgement), THe participating judge judging Group 2 (judging other topic), the neutral judge judging group 1, and the neutral judge judging group 2. This evaluation was conducting for each debating model: i.e. Claude 3, Mistral AI, and GPT4. The results are displayed in the figure below on the left column.

With this four groups we can compute a relative score as the difference between the score of participating judge and the score of the neutral judge for each group. When this relative score is greater for the first group (the answer to the question: does the topic has been generated by the participating AI is True ) than for the second group (resp. False). The results are displayed in the right column of the figure below. The box plot for group 1 and group 2 looks alike. Therefore it seems that the AI don't perform better on their own topic. 

In [ ]:
judgements["is_team_topic_creator"] = judgements["team_entity"] == judgements["topic_creator_entity"]
boxes = []
for judge_of_interest in teams_of_interests:
    box = plot_analyses(judgements, judge_of_interest = judge_of_interest, columngroup = 'is_team_topic_creator')
    boxes.append(box)
VBox(boxes)

## Which team, proposing or opposing, are LLMs better suited for?

The debate consists of four rounds of argumentation per team, be it the proposing or the opposing team, each offering distinct advantages and disadvantages. The proposing team initiates the debate, framing it with their initial arguments. They may corner the opponent early on, potentially leading to a clear victory if the opposing team fails to counter effectively. Conversely, the opposing team, which concludes the debate, often has a better chance of winning, as the last word tends to carry more weight and influence. To assess these possibilities, LLM scores were grouped based on their team role (proposing or opposing). Since all LLMs evaluated all debates, potential biases from judging AI were assumed to cancel out.

Upon analyzing the data, it appears that Claude-3 and GPT-4 perform better when placed in the opposing team, while Mistral-AI perform better when in the proposing team.

In [ ]:
judgements["is_team_proposing"] = judgements["team_id"] == judgements["team_proposing_id"]
judgements.head()
boxes = []
for team_of_interest in teams_of_interests:
    fig = px.box(judgements.loc[judgements.team_entity==team_of_interest], x="Categories", y="Normalised_score", color="is_team_proposing")
    fw1 = go.FigureWidget(data=fig.data)
    fw1.update_layout(
            title_text=f"<b>{team_of_interest}</b><br>Scores on proposing/opposing roles",
            yaxis_title='Relative Score',
            boxmode='group' # group together boxes of the different traces for each value of x
        )
    boxes.append(fw1)
HBox(boxes)

## Do they impartially assess debates, refraining from awarding higher scores to themselves?

The question of impartiality in judging debates, specifically in regard to whether participants assign higher scores to their own team, is a critical aspect of evaluating the integrity of the process. It's essential to ensure that judgments are based solely on the quality of arguments presented, rather than any bias towards one's own team. Fairness hinges on judges' ability to maintain objectivity and render judgments without favoritism. If judges consistently award higher scores to their own team, it undermines the credibility of the entire debate, casting doubt on the validity of the outcomes. Therefore, establishing and upholding standards of fairness in judging is paramount to the integrity and effectiveness of the debating process.

To assess for the eventualilty of unfairness, the score of the participating judge and a neutral judge (in this case Gemini) were compared for group 1: the participating judge evaluated their own discourse, and group 2: the partipating judge didn't rated the discourse of others LLMs. The results of the four groups: participating judge juding group 1 (P1), group 2 (P2), the neutral judge juding group 1 (N1), and group 2 (N2) are presented on the left column in the result below. An unfair judgement implies that (P1-N1)>(P2-N2). The difference between the judge scores for each group are presented on the right column in the figure below. We can see that Claude-3 has a marginal tendence to grade its own debate better (True/False correspond to the answer of the question: are the participating judge has been grading its own debate). 

In [ ]:
judgements["is_team_self_judging"] = judgements["team_entity"] == judgements["judge_entity"]
boxes = []
for judge_of_interest in teams_of_interests:
    box = plot_analyses(judgements, judge_of_interest = judge_of_interest, columngroup = 'is_team_self_judging')
    boxes.append(box)
VBox(boxes)

## Which model is best at debating?

Perhaps the most awaited answer question for the one interested in using LLMs for training for a debate or assisting during one. As we saw in the overview, we did not have a clear winner. However, as we saw above the AI may be bias toward their own topic, or judge their debate better than the one of other participants, i.e. not playing fair. The bias on their own topic is not relevant, as each AI debated on their own as well as on their opponent topic in a balanced topic, and it did not seem to play a role. However Claude-3 seemed to evaluate itself better than other model.

To decide which model is best at debating, I choose to evaluate their performance based on the neutral judge. This judge was indeed used for all debates and did not participate in a competitions. By looking at the normalised score of the neutral judge for the competions of GPT-4 vs Mistral AI it is not evident than one or the other performs better. However when looking at the score for the competition between Mistral AI and Claude-3 it seems that the later is better at debating (at least as seen from the prompts of Gemini).

In [ ]:
competitors = ['OpenAIChatter|gpt-4', 'MistralAIChatter|mistral-large-latest']
condition = judgements.opposing_entity.isin(competitors)
condition &= judgements.proposing_entity.isin(competitors)
condition &= judgements.judge_entity.isin([neutral_judge])
selected_judegements = judgements.loc[condition]

fig = px.box(selected_judegements, x="Categories", y="Normalised_score", color="team_entity")
fw1 = go.FigureWidget(data=fig.data)
fw1.update_layout(
        title_text=f"<b>Competitions:</b><br>{competitors[0]} vs {competitors[1]}",
        yaxis_title='Normalised Score',
        boxmode='group' # group together boxes of the different traces for each value of x
    )

competitors = ['Claude3AiChatter|claude-3-opus-20240229', 'MistralAIChatter|mistral-large-latest']
condition = judgements.opposing_entity.isin(competitors)
condition &= judgements.proposing_entity.isin(competitors)
condition &= judgements.judge_entity.isin([neutral_judge])
selected_judegements = judgements.loc[condition]

fig = px.box(selected_judegements, x="Categories", y="Normalised_score", color="team_entity")
fw2 = go.FigureWidget(data=fig.data)
fw2.update_layout(
        title_text=f"<b>Competitions:</b><br>{competitors[0]} vs {competitors[1]}",
        yaxis_title='Normalised Score',
        boxmode='group' # group together boxes of the different traces for each value of x
    )
HBox([fw1, fw2])

## Do LLMs exhibit a propensity to alter their opinions? 

Decision-making processes often display biases towards initial, final decisions, or one own decision, a phenomenon observed in both human and potentially LLM decision-making. To assess this possibility, each LLM was presented with the discourse followed by various judgments and asked to select one. In essence, they acted akin to a public audience voting after a jury verdict. Since judgments were rendered by LLMs, it became possible to examine whether voting LLMs displayed a preference for their own judgments by tallying the number of times they chose an LLM's judgment. The data revealed a notable preference for Gemini's judgments, unexpectedly high and suggestive of a bias towards the order in which judgments were presented, which, incidentally, was not randomized.

In [ ]:
public_voting = result_manager.load_votes_for_analyses()
toplot = public_voting.groupby(["public_model_entity","voted_for_judgement_model_entity"]).voted_for_judgement_id.count()
toplot.name='Count'
total = toplot.groupby("public_model_entity").sum()
toplot/=total
toplot*=100
toplot.name = 'Percent'
toplot = toplot.reset_index()
color_discrete_map = {k:v for k,v in zip(toplot.voted_for_judgement_model_entity.unique(), px.colors.qualitative.Plotly)}
color_discrete_map["Failed to vote"] = "gray"
fig = px.bar(toplot, x="public_model_entity", y="Percent", color="voted_for_judgement_model_entity",barmode='group', color_discrete_map=color_discrete_map)
fig

As the judgments were not randomized in their order, the observed pattern might stem from a bias towards the placement of arguments, possibly favoring those presented first or last in the prompt. To investigate this preference, we can analyze how frequently an LLM voted for each judgment. Claude-3, MistralAI, and GPT-4 notably favor judgments seen first, while Gemini exhibits a preference for either the first or last judgment.

In [ ]:
toplot = public_voting.groupby(["public_model_entity","voted_for"]).voted_for_judgement_id.count()
toplot.name='Count'
total = toplot.groupby("public_model_entity").sum()
toplot/=total
toplot*=100
toplot.name = 'Percent'
toplot = toplot.reset_index()
color_discrete_map = {k:v for k,v in zip(toplot.voted_for.unique(), px.colors.qualitative.Plotly)}
color_discrete_map["Failed to vote"] = "gray"
px.bar(toplot, x="public_model_entity", y="Percent", color="voted_for",barmode='group', color_discrete_map=color_discrete_map)

The decision made by the AI was following a forced choice design, in which the participant has to select one of the provided options. A forced choice may however arbitrary when the provided choices are equivalent. We can quantatitely assess for the equivalence of the different judgement by looking at the correlation between the score across LLMs. In the Scatterplot Matrix below the scores given by each LLMs for each discourse are shown. The color indicate the associated categories of the score. Note that the Score and not the normalised score are ploted, because the LLMs were given the former and not the later during the public voting. We see that the score are not really correleted meaning that the judgement are potentially differing and that the bias choice made by the AI are not simply explained by a quantitative correlation between the scores. 

In [ ]:
toplot = judgements.set_index(["judge_entity", "discourse_id", "team_entity","Categories"]).unstack(0).Score
toplot.columns = [c.replace('|','<br>') for c in  toplot.columns ]
dimensions = toplot.columns
toplot = toplot.reset_index()
toplot = toplot.loc[:, [*dimensions, 'Categories']]
fig = create_scatterplotmatrix(toplot, index='Categories', diag='box', height=1000, width=1000)
fig.update_layout(legend={"title":"Categories"})
fig.update_layout(legend={"title":"Categories"})  
fig.show()

## Debate examples

In the preceding analyses, we delved into the world of LLMs without considering the qualities of the narratives, arguments, and debaters from a human perspective. The evaluation was done from prompt to prompt. While it may be impractical to meticulously evaluate every debate, examining a few examples could offer intriguing insights. Consequently, I opted to look at two cases: the debate boasting the highest overall score and the debate with the most differing scores between the proposing and opposing teams.

### The highest rated debate

In [ ]:
best_debate = judgements.groupby('discourse_id').Normalised_score.sum().idxmax()
discourse = result_manager.load_discourse(discourse_id = best_debate)
res = judgements.loc[judgements.discourse_id==best_debate].set_index(["team_entity","Categories","judge_entity"])
model_proposing = res.proposing_entity.unique()[0]
model_opposing = res.opposing_entity.unique()[0]
res = res.Normalised_score*100
res = res.reset_index()
discourse['is_proposing'] = discourse["model_speaking_entity"] == model_proposing

sql_statement = \
f"""
SELECT
    topic.Subject,
    topic.Rational
FROM 'dim_discourse' AS dim
LEFT JOIN 'topics' as topic
    USING (topic_id)
WHERE dim.discourse_id = "{best_debate}"
"""
topic = pd.read_sql(sql_statement, result_manager.connection).iloc[0]
topic

In [ ]:
fig = px.scatter(res, x='Normalised_score', y='Categories', color='judge_entity', facet_col="team_entity")
fig.update_traces(marker=dict(size=12))
fig.update_layout(title_text=f"<b>Topic:{topic.Subject}</b>: {model_proposing.split('|')[0]} proposed / {model_opposing.split('|')[0]} opposed ")

In [ ]:
from IPython.display import HTML as html_print

def format_argument(arg, color='black'):
    color = "green" if arg.is_proposing else "tomato"
    space = "</br>" if arg.is_proposing else ""
    tag = "p"
    text = f"{space}<i>({arg.model_speaking_entity.split('|')[0]})</i>: {arg.Argument}"
    return f"<{tag} align='justify'>{text}</{tag}>"

In [ ]:
to_print = f"<b> {topic.Subject} </b>"
to_print += f"<p> {topic.Rational} </p>"

for _, arg in discourse.sort_values(by='ith_argument').iterrows():
    to_print+=format_argument(arg)
html_print(to_print)

### The clearest win/lost 

In [ ]:
judgements["is_team_proposing"] = judgements["team_id"] == judgements["team_proposing_id"]
contrasted_debate = judgements.set_index(['discourse_id', 'is_team_proposing', 'Categories', 'judge_entity']).Normalised_score.unstack(1).diff(axis=1).dropna(axis=1).groupby('discourse_id').sum().abs().idxmax()
contrasted_debate = contrasted_debate.values[0]

In [ ]:
discourse = result_manager.load_discourse(discourse_id = contrasted_debate)
res = judgements.loc[judgements.discourse_id==contrasted_debate].set_index(["team_entity","Categories","judge_entity"])
model_proposing = res.proposing_entity.unique()[0]
model_opposing = res.opposing_entity.unique()[0]
res = res.Normalised_score*100
res = res.reset_index()
discourse['is_proposing'] = discourse["model_speaking_entity"] == model_proposing

sql_statement = \
f"""
SELECT
    topic.Subject,
    topic.Rational
FROM 'dim_discourse' AS dim
LEFT JOIN 'topics' as topic
    USING (topic_id)
WHERE dim.discourse_id = '{contrasted_debate}'
"""
topic = pd.read_sql(sql_statement, result_manager.connection).iloc[0]
topic

In [ ]:
fig = px.scatter(res, x='Normalised_score', y='Categories', color='judge_entity', facet_col="team_entity")
fig.update_traces(marker=dict(size=12))
fig.update_layout(title_text=f"<b>Topic:{topic.Subject}</b>")

In [ ]:
to_print = f"<b> {topic.Subject} </b>"
to_print += f"<p> {topic.Rational} </p>"

for _, arg in discourse.sort_values(by='ith_argument').iterrows():
    to_print+=format_argument(arg)
html_print(to_print)